In [1]:
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import polars as pl
import seaborn as sns
import scipy
from teeplot import teeplot as tp


In [2]:
teeplot_subdir = "adaptive-evolution-rate-nodirect.ipynb"


In [3]:
def load_and_process(url: str, mutation: str) -> pl.DataFrame:
    df = pl.read_parquet(
        url,
        use_pyarrow=True,
    ).with_columns(
        pl.col("SLIP_INSERTION_BOOL_MASK").any().over(
            ["Treatment", "Run ID", "Generation Born"],
        ).alias("SLIP_INSERTION_BOOL_MASK any"),
    ).group_by(
        ["Task", "Treatment", "Run ID", "Generation Born"],
    ).first().with_columns(
        pl.col("Task").replace_strict(
            {
                "AND": 2,
                "ANDNOT": 3,
                "NAND": 1,
                "NOR": 4,
                "NOT": 1,
                "OR": 3,
                "ORNOT": 2,
                "XOR": 4,
                "EQUALS": 5,
            },
        ).alias("Components"),
        pl.lit(mutation).alias("Mutation"),
    )

    return df


In [4]:
df1 = load_and_process("https://osf.io/emh23/download", "poisson")


sys:1: CategoricalRemappingWarning: Local categoricals have different encodings, expensive re-encoding is done to perform this merge operation. Consider using a StringCache or an Enum type if the categories are known in advance


In [5]:
df2 = load_and_process("https://osf.io/gk2ty/download", "per site")


In [6]:
df = pl.concat([df1, df2])
df


Task,Treatment,Run ID,Generation Born,Genome Length,Lineage Generation Index,Is Task Coding Site,Is Viability Site,Is Task Coding Site Cumulative Count,Is Viability Site Cumulative Count,Prev Is Task Coding Site Cumulative Count,Prev Is Viability Site Cumulative Count,Prev Is Task Coding Site,Prev Is Viability Site,Site,Has Task,has task,delta has task,Delta Has Task,CHILD_SOURCE_MAP,POINT_MUTATION_BOOL_MASK,SLIP_INSERTION_ORIGIN_BOOL_MASK,SLIP_INSERTION_RESULT_BOOL_MASK,GENOME_CHARACTERS,Is Task Coding Site Delta,Is Viability Site Delta,Point Mutation Cumulative Count,Slip Insertion Cumulative Count,Prev Slip Insertion Cumulative Count,SLIP_INSERTION_BOOL_MASK,Update Born,SLIP_INSERTION_BOOL_MASK any,Components,Mutation
cat,cat,cat,u16,i64,u16,bool,bool,i16,i16,i16,i16,bool,bool,u16,bool,bool,i8,i8,i16,bool,bool,bool,cat,i8,i8,u16,u16,u16,bool,i32,bool,i64,str
"""NOT""","""Slip-""","""run_1807""",1080,104,66,false,true,0,67,0,66,false,true,0,true,true,0,0,0,false,false,false,"""w""",0,0,0,0,0,false,8036,false,1,"""poisson"""
"""NOR""","""Slip-""","""run_1813""",934,101,67,false,true,0,68,0,67,false,true,0,false,false,0,0,0,false,false,false,"""w""",0,0,0,0,0,false,7619,false,4,"""poisson"""
"""NAND""","""Slip-""","""run_1823""",1513,106,74,false,true,0,75,0,74,false,true,0,true,true,0,0,0,false,false,false,"""w""",0,0,0,0,0,false,13268,false,1,"""poisson"""
"""EQUALS""","""Slip+""","""run_1835""",435,772,47,false,true,0,48,0,47,false,true,0,false,false,0,0,0,false,false,false,"""w""",0,0,0,0,0,false,4734,true,5,"""poisson"""
"""OR""","""Slip+""","""run_1855""",447,725,40,false,true,0,41,0,40,false,true,0,true,true,0,0,0,false,false,false,"""w""",0,0,0,0,0,false,11233,false,3,"""poisson"""
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""NAND""","""Slip-duplicate""","""run_1601""",1816,218,199,false,true,0,200,0,199,false,true,0,true,true,0,0,0,false,false,false,"""w""",0,0,0,0,0,false,29029,true,1,"""per site"""
"""NOR""","""Slip-duplicate""","""run_1601""",2191,218,235,false,true,0,236,0,235,false,true,0,true,true,0,0,0,false,false,false,"""w""",0,0,0,0,0,false,36199,false,4,"""per site"""
"""ANDNOT""","""Long-Ancestor-Control-Treatmen…","""run_1741""",124,1000,53,false,true,0,54,0,53,false,true,0,false,false,0,0,0,false,false,false,"""w""",0,0,0,0,0,false,10391,false,3,"""per site"""


In [7]:
df = df.with_columns(
    (
        pl.col("Generation Born")
        + 0 / pl.col("has task")
    ).min().over(
        ["Treatment", "Run ID", "Mutation", "Task"],
    ).alias("First Task Generation"),
    pl.col("Generation Born").max().over(
        ["Treatment", "Run ID", "Mutation"]
    ).alias("Last Generation"),
)
df


Task,Treatment,Run ID,Generation Born,Genome Length,Lineage Generation Index,Is Task Coding Site,Is Viability Site,Is Task Coding Site Cumulative Count,Is Viability Site Cumulative Count,Prev Is Task Coding Site Cumulative Count,Prev Is Viability Site Cumulative Count,Prev Is Task Coding Site,Prev Is Viability Site,Site,Has Task,has task,delta has task,Delta Has Task,CHILD_SOURCE_MAP,POINT_MUTATION_BOOL_MASK,SLIP_INSERTION_ORIGIN_BOOL_MASK,SLIP_INSERTION_RESULT_BOOL_MASK,GENOME_CHARACTERS,Is Task Coding Site Delta,Is Viability Site Delta,Point Mutation Cumulative Count,Slip Insertion Cumulative Count,Prev Slip Insertion Cumulative Count,SLIP_INSERTION_BOOL_MASK,Update Born,SLIP_INSERTION_BOOL_MASK any,Components,Mutation,First Task Generation,Last Generation
cat,cat,cat,u16,i64,u16,bool,bool,i16,i16,i16,i16,bool,bool,u16,bool,bool,i8,i8,i16,bool,bool,bool,cat,i8,i8,u16,u16,u16,bool,i32,bool,i64,str,f64,u16
"""NOT""","""Slip-""","""run_1807""",1080,104,66,false,true,0,67,0,66,false,true,0,true,true,0,0,0,false,false,false,"""w""",0,0,0,0,0,false,8036,false,1,"""poisson""",269.0,7296
"""NOR""","""Slip-""","""run_1813""",934,101,67,false,true,0,68,0,67,false,true,0,false,false,0,0,0,false,false,false,"""w""",0,0,0,0,0,false,7619,false,4,"""poisson""",NaN,7434
"""NAND""","""Slip-""","""run_1823""",1513,106,74,false,true,0,75,0,74,false,true,0,true,true,0,0,0,false,false,false,"""w""",0,0,0,0,0,false,13268,false,1,"""poisson""",397.0,6192
"""EQUALS""","""Slip+""","""run_1835""",435,772,47,false,true,0,48,0,47,false,true,0,false,false,0,0,0,false,false,false,"""w""",0,0,0,0,0,false,4734,true,5,"""poisson""",814.0,1160
"""OR""","""Slip+""","""run_1855""",447,725,40,false,true,0,41,0,40,false,true,0,true,true,0,0,0,false,false,false,"""w""",0,0,0,0,0,false,11233,false,3,"""poisson""",197.0,962
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""NAND""","""Slip-duplicate""","""run_1601""",1816,218,199,false,true,0,200,0,199,false,true,0,true,true,0,0,0,false,false,false,"""w""",0,0,0,0,0,false,29029,true,1,"""per site""",319.0,3031
"""NOR""","""Slip-duplicate""","""run_1601""",2191,218,235,false,true,0,236,0,235,false,true,0,true,true,0,0,0,false,false,false,"""w""",0,0,0,0,0,false,36199,false,4,"""per site""",906.0,3031
"""ANDNOT""","""Long-Ancestor-Control-Treatmen…","""run_1741""",124,1000,53,false,true,0,54,0,53,false,true,0,false,false,0,0,0,false,false,false,"""w""",0,0,0,0,0,false,10391,false,3,"""per site""",315.0,658


In [8]:
df.group_by(
    ["Treatment", "Mutation"],
).agg(
    pl.col("Last Generation").min(),
)


Treatment,Mutation,Last Generation
cat,str,u16
"""Slip-""","""poisson""",2050
"""Baseline-Treatment""","""per site""",4081
"""Slip-duplicate""","""per site""",901
"""Slip+""","""poisson""",573
"""Slip-_Long""","""poisson""",340
"""Long-Ancestor-Control-Treatmen…","""per site""",538


In [9]:
least_last_generation = 600
least_last_generation


600

In [10]:
dfz = df.filter(
    # (pl.col("Generation Born") <= least_last_generation)
    pl.col("has task")
    & (pl.col("delta has task") == 1)
    & (pl.col("First Task Generation") == pl.col("Generation Born"))
    & (
        (pl.col("Treatment") != "Slip-duplicate")
        | (~pl.col("SLIP_INSERTION_BOOL_MASK any"))
    )
).group_by(
    ["Task", "Treatment", "Run ID", "Components", "Mutation"],
).agg(
    pl.col("Generation Born").min(),
)
dfz


Task,Treatment,Run ID,Components,Mutation,Generation Born
cat,cat,cat,i64,str,u16
"""OR""","""Slip-""","""run_1807""",3,"""poisson""",1594
"""AND""","""Long-Ancestor-Control-Treatmen…","""run_1760""",2,"""per site""",88
"""AND""","""Slip+""","""run_1849""",2,"""poisson""",207
"""ORNOT""","""Slip-_Long""","""run_1799""",2,"""poisson""",219
"""NAND""","""Slip+""","""run_1840""",1,"""poisson""",91
…,…,…,…,…,…
"""OR""","""Slip+""","""run_1853""",3,"""poisson""",449
"""NOR""","""Slip-""","""run_1805""",4,"""poisson""",782
"""ORNOT""","""Baseline-Treatment""","""run_1559""",2,"""per site""",266


In [11]:
dfz


Task,Treatment,Run ID,Components,Mutation,Generation Born
cat,cat,cat,i64,str,u16
"""OR""","""Slip-""","""run_1807""",3,"""poisson""",1594
"""AND""","""Long-Ancestor-Control-Treatmen…","""run_1760""",2,"""per site""",88
"""AND""","""Slip+""","""run_1849""",2,"""poisson""",207
"""ORNOT""","""Slip-_Long""","""run_1799""",2,"""poisson""",219
"""NAND""","""Slip+""","""run_1840""",1,"""poisson""",91
…,…,…,…,…,…
"""OR""","""Slip+""","""run_1853""",3,"""poisson""",449
"""NOR""","""Slip-""","""run_1805""",4,"""poisson""",782
"""ORNOT""","""Baseline-Treatment""","""run_1559""",2,"""per site""",266


In [12]:
maxgen = dfz["Generation Born"].max()

records = []
records2 = set()
for row in dfz.iter_rows(named=True):
    records.append(
        pd.DataFrame(
            {
                "Task": row["Task"],
                "Treatment": row["Treatment"],
                "Run ID": row["Run ID"],
                "Generation": np.arange(maxgen),
                "Components": row["Components"],
                "Has Task":  row["Generation Born"] <= np.arange(maxgen),
                "mutation": row["Mutation"],
            },
        ),
    )
    records2.add(
        (row["Run ID"], row["Treatment"], row["Task"], row["Components"], row["Mutation"]),
    )

for v, grp in df.to_pandas().groupby(
    ["Run ID", "Treatment", "Task", "Components", "Mutation"],
    observed=True,
):
    if v not in records2:
        print(v)
        records.append(
            pd.DataFrame(
                {
                    "Task": v[2],
                    "Treatment": v[1],
                    "Run ID": v[0],
                    "Components": v[3],
                    "Generation": np.arange(maxgen),
                    "Has Task":  False,
                    "mutation": v[4],
                },
            ),
        )

dfz_ = pd.concat(records, ignore_index=True)


('run_1770', 'Slip-_Long', 'ANDNOT', np.int64(3), 'poisson')
('run_1770', 'Slip-_Long', 'EQUALS', np.int64(5), 'poisson')
('run_1770', 'Slip-_Long', 'NOR', np.int64(4), 'poisson')
('run_1770', 'Slip-_Long', 'XOR', np.int64(4), 'poisson')
('run_1771', 'Slip-_Long', 'AND', np.int64(2), 'poisson')
('run_1771', 'Slip-_Long', 'ANDNOT', np.int64(3), 'poisson')
('run_1771', 'Slip-_Long', 'EQUALS', np.int64(5), 'poisson')
('run_1771', 'Slip-_Long', 'NOR', np.int64(4), 'poisson')
('run_1771', 'Slip-_Long', 'OR', np.int64(3), 'poisson')
('run_1771', 'Slip-_Long', 'XOR', np.int64(4), 'poisson')
('run_1772', 'Slip-_Long', 'AND', np.int64(2), 'poisson')
('run_1772', 'Slip-_Long', 'ANDNOT', np.int64(3), 'poisson')
('run_1772', 'Slip-_Long', 'EQUALS', np.int64(5), 'poisson')
('run_1772', 'Slip-_Long', 'NOR', np.int64(4), 'poisson')
('run_1772', 'Slip-_Long', 'NOT', np.int64(1), 'poisson')
('run_1772', 'Slip-_Long', 'OR', np.int64(3), 'poisson')
('run_1772', 'Slip-_Long', 'XOR', np.int64(4), 'poisson'

('run_1839', 'Slip+', 'XOR', np.int64(4), 'poisson')
('run_1840', 'Slip+', 'XOR', np.int64(4), 'poisson')
('run_1841', 'Slip+', 'EQUALS', np.int64(5), 'poisson')
('run_1841', 'Slip+', 'NOR', np.int64(4), 'poisson')
('run_1842', 'Slip+', 'EQUALS', np.int64(5), 'poisson')
('run_1842', 'Slip+', 'NOR', np.int64(4), 'poisson')
('run_1842', 'Slip+', 'OR', np.int64(3), 'poisson')
('run_1842', 'Slip+', 'XOR', np.int64(4), 'poisson')
('run_1843', 'Slip+', 'EQUALS', np.int64(5), 'poisson')
('run_1843', 'Slip+', 'XOR', np.int64(4), 'poisson')
('run_1844', 'Slip+', 'EQUALS', np.int64(5), 'poisson')
('run_1844', 'Slip+', 'XOR', np.int64(4), 'poisson')
('run_1845', 'Slip+', 'EQUALS', np.int64(5), 'poisson')
('run_1845', 'Slip+', 'XOR', np.int64(4), 'poisson')
('run_1848', 'Slip+', 'XOR', np.int64(4), 'poisson')
('run_1849', 'Slip+', 'EQUALS', np.int64(5), 'poisson')
('run_1851', 'Slip+', 'EQUALS', np.int64(5), 'poisson')
('run_1851', 'Slip+', 'XOR', np.int64(4), 'poisson')
('run_1852', 'Slip+', 'EQU

('run_1615', 'Slip-duplicate', 'ORNOT', np.int64(2), 'per site')
('run_1616', 'Slip-duplicate', 'NAND', np.int64(1), 'per site')
('run_1617', 'Slip-duplicate', 'EQUALS', np.int64(5), 'per site')
('run_1617', 'Slip-duplicate', 'NAND', np.int64(1), 'per site')
('run_1618', 'Slip-duplicate', 'NOR', np.int64(4), 'per site')
('run_1618', 'Slip-duplicate', 'XOR', np.int64(4), 'per site')
('run_1619', 'Slip-duplicate', 'NOR', np.int64(4), 'per site')
('run_1740', 'Long-Ancestor-Control-Treatment', 'EQUALS', np.int64(5), 'per site')
('run_1740', 'Long-Ancestor-Control-Treatment', 'XOR', np.int64(4), 'per site')
('run_1741', 'Long-Ancestor-Control-Treatment', 'EQUALS', np.int64(5), 'per site')
('run_1741', 'Long-Ancestor-Control-Treatment', 'XOR', np.int64(4), 'per site')
('run_1742', 'Long-Ancestor-Control-Treatment', 'EQUALS', np.int64(5), 'per site')
('run_1742', 'Long-Ancestor-Control-Treatment', 'XOR', np.int64(4), 'per site')
('run_1743', 'Long-Ancestor-Control-Treatment', 'EQUALS', np.in

teeplots/adaptive-evolution-rate-nodirect.ipynb/col=task+errorbar=ci+hue=treatment+kind=line+post=plt-xlim-0-600+row=mutation+viz=relplot+x=generation+y=has-task+ext=.pdf


teeplots/adaptive-evolution-rate-nodirect.ipynb/col=task+errorbar=ci+hue=treatment+kind=line+post=plt-xlim-0-600+row=mutation+viz=relplot+x=generation+y=has-task+ext=.png


In [13]:
tp.tee(
    sns.relplot,
    data=dfz_[
        dfz_["Generation"] % 16 == 0
    ].reset_index(),
    col="Task",
    row="mutation",
    x="Generation",
    y="Has Task",
    hue="Treatment",
    kind="line",
    # teeplot_postprocess='plt.xscale("log")',
    teeplot_postprocess=f'plt.xlim(0, {least_last_generation})',
    teeplot_subdir=teeplot_subdir,
    errorbar="ci",
)


teeplots/adaptive-evolution-rate-nodirect.ipynb/col=components+errorbar=ci+hue=treatment+kind=line+post=plt-xlim-0-600+row=mutation+viz=relplot+x=generation+y=has-task+ext=.pdf


teeplots/adaptive-evolution-rate-nodirect.ipynb/col=components+errorbar=ci+hue=treatment+kind=line+post=plt-xlim-0-600+row=mutation+viz=relplot+x=generation+y=has-task+ext=.png


In [14]:
tp.tee(
    sns.relplot,
    data=dfz_[
        dfz_["Generation"] % 16 == 0
    ].reset_index(),
    col="Components",
    row="mutation",
    x="Generation",
    y="Has Task",
    hue="Treatment",
    kind="line",
    teeplot_postprocess=f"plt.xlim(0, {least_last_generation})",
    teeplot_subdir=teeplot_subdir,
    errorbar="ci",
)


In [15]:
fil = dfz_[
    (dfz_["Generation"] == 599)
]

for task in fil["Task"].unique():
    tab = [
        [
            (
                (fil["Treatment"] == "Baseline-Treatment")
                & (fil["Task"] == task)
                & fil["Has Task"]
            ).sum(),
            (
                (fil["Treatment"] == "Slip-duplicate")
                & (fil["Task"] == task)
                & fil["Has Task"]
            ).sum(),
        ],
        [
            (
                (fil["Treatment"] == "Baseline-Treatment")
                & (fil["Task"] == task)
                & (~fil["Has Task"])
            ).sum(),
            (
                (fil["Treatment"] == "Slip-duplicate")
                & (fil["Task"] == task)
                & (~fil["Has Task"])
            ).sum(),
        ],
    ]
    print(task, tab)
    print("    ", scipy.stats.fisher_exact(tab))


OR [[np.int64(16), np.int64(20)], [np.int64(14), np.int64(10)]]
     SignificanceResult(statistic=np.float64(0.5714285714285714), pvalue=np.float64(0.4295691931947636))
AND [[np.int64(8), np.int64(23)], [np.int64(22), np.int64(7)]]
     SignificanceResult(statistic=np.float64(0.11067193675889328), pvalue=np.float64(0.00023089337753119202))
ORNOT [[np.int64(29), np.int64(13)], [np.int64(1), np.int64(17)]]
     SignificanceResult(statistic=np.float64(37.92307692307692), pvalue=np.float64(7.954964600623303e-06))
NAND [[np.int64(30), np.int64(16)], [np.int64(0), np.int64(14)]]
     SignificanceResult(statistic=inf, pvalue=np.float64(1.6767384375519427e-05))
ANDNOT [[np.int64(11), np.int64(21)], [np.int64(19), np.int64(9)]]
     SignificanceResult(statistic=np.float64(0.24812030075187969), pvalue=np.float64(0.019096056845063346))
EQUALS [[np.int64(0), np.int64(5)], [np.int64(30), np.int64(25)]]
     SignificanceResult(statistic=np.float64(0.0), pvalue=np.float64(0.05218554861730598))
NOR [[

In [16]:
fil = dfz_[
    (dfz_["Generation"] == 599)
]

for task in fil["Task"].unique():
    tab = [
        [
            (
                (fil["Treatment"] == "Long-Ancestor-Control-Treatment")
                & (fil["Task"] == task)
                & fil["Has Task"]
            ).sum(),
            (
                (fil["Treatment"] == "Slip-duplicate")
                & (fil["Task"] == task)
                & fil["Has Task"]
            ).sum(),
        ],
        [
            (
                (fil["Treatment"] == "Long-Ancestor-Control-Treatment")
                & (fil["Task"] == task)
                & (~fil["Has Task"])
            ).sum(),
            (
                (fil["Treatment"] == "Slip-duplicate")
                & (fil["Task"] == task)
                & (~fil["Has Task"])
            ).sum(),
        ],
    ]
    print(task, tab)
    print("    ", scipy.stats.fisher_exact(tab))


OR [[np.int64(30), np.int64(20)], [np.int64(0), np.int64(10)]]
     SignificanceResult(statistic=inf, pvalue=np.float64(0.0007970131314101389))
AND [[np.int64(30), np.int64(23)], [np.int64(0), np.int64(7)]]
     SignificanceResult(statistic=inf, pvalue=np.float64(0.010542535074203227))
ORNOT [[np.int64(30), np.int64(13)], [np.int64(0), np.int64(17)]]
     SignificanceResult(statistic=inf, pvalue=np.float64(6.185596344065137e-07))
NAND [[np.int64(30), np.int64(16)], [np.int64(0), np.int64(14)]]
     SignificanceResult(statistic=inf, pvalue=np.float64(1.6767384375519427e-05))
ANDNOT [[np.int64(30), np.int64(21)], [np.int64(0), np.int64(9)]]
     SignificanceResult(statistic=inf, pvalue=np.float64(0.001935603319138909))
EQUALS [[np.int64(2), np.int64(5)], [np.int64(28), np.int64(25)]]
     SignificanceResult(statistic=np.float64(0.35714285714285715), pvalue=np.float64(0.4238099099829697))
NOR [[np.int64(22), np.int64(18)], [np.int64(8), np.int64(12)]]
     SignificanceResult(statistic=np.

In [17]:
fil = dfz_[
    (dfz_["Generation"] == 599)
]

for components in sorted(fil["Components"].unique()):
    tab = [
        [
            (
                (fil["Treatment"] == "Long-Ancestor-Control-Treatment")
                & (fil["Components"] == components)
                & fil["Has Task"]
            ).sum(),
            (
                (fil["Treatment"] == "Slip-duplicate")
                & (fil["Components"] == components)
                & fil["Has Task"]
            ).sum(),
        ],
        [
            (
                (fil["Treatment"] == "Long-Ancestor-Control-Treatment")
                & (fil["Components"] == components)
                & (~fil["Has Task"])
            ).sum(),
            (
                (fil["Treatment"] == "Slip-duplicate")
                & (fil["Components"] == components)
                & (~fil["Has Task"])
            ).sum(),
        ],
    ]
    print(components, tab)
    print("    ", scipy.stats.fisher_exact(tab))


1 [[np.int64(60), np.int64(36)], [np.int64(0), np.int64(24)]]
     SignificanceResult(statistic=inf, pvalue=np.float64(6.632030256515911e-09))
2 [[np.int64(60), np.int64(36)], [np.int64(0), np.int64(24)]]
     SignificanceResult(statistic=inf, pvalue=np.float64(6.632030256515911e-09))
3 [[np.int64(60), np.int64(41)], [np.int64(0), np.int64(19)]]
     SignificanceResult(statistic=inf, pvalue=np.float64(7.009823797510181e-07))
4 [[np.int64(24), np.int64(23)], [np.int64(36), np.int64(37)]]
     SignificanceResult(statistic=np.float64(1.0724637681159421), pvalue=1.0)
5 [[np.int64(2), np.int64(5)], [np.int64(28), np.int64(25)]]
     SignificanceResult(statistic=np.float64(0.35714285714285715), pvalue=np.float64(0.4238099099829697))
